### Enter full names of group members:

##### Name A: Kristian Sørli

In [2]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [3]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [ ]:
def dgim_algorithm(stream_path, N):
    
    # Create the buckets and initialize the timestamp
    bucket_list = []
    timestamp = 0
    end_time_stamp = -1

    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break

            timestamp += 1

            if bit == '1':
                # Insert a new bucket of size 1 with this timestamp
                if len(bucket_list) == 0:
                    bucket_list.append([timestamp])
                else:
                    bucket_list[0].insert(0, timestamp) 

                end_time_stamp = timestamp  

                # Merge buckets if needed
                i = 0
                while i < len(bucket_list):
                    if len(bucket_list[i]) > 2:
                        if i + 1 >= len(bucket_list):
                            bucket_list.append([])  
                        b1 = bucket_list[i].pop()  
                        b2 = bucket_list[i].pop()  
                        merged_time = b2  
                        bucket_list[i + 1].insert(0, merged_time)
                        i += 1  
                    else:
                        break  

            # Remove outdated buckets 
            boundary = timestamp - N
            for size_index, bucket in enumerate(bucket_list):
                size = 2 ** size_index
                while bucket and bucket[-1] <= boundary:
                    bucket.pop()

    return bucket_list, end_time_stamp


In [5]:
bucket = dgim_algorithm(stream_path, N)

In [6]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[1010099], [1010096, 1010091], [1010089, 1010083], [1010075, 1010063], [1010044], [1010006], [1009946, 1009821], [1009688]]
The end timestamp: 1010099


#### 1.2. Query the Bucket 

In [7]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [ ]:
def dgim_query(bucket, N, k):  
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
    
    one_count = 0

    window_start = end_time_stamp - k + 1

    halved = False

    if k > N:
        raise ValueError(f"k must be less than or equal to N, but got k={k} and N={N}.")

    for i, buckets in enumerate(bucket_list): ## Iterate through the buckets
        size = 2 ** i  ## Size of the bucket
        if len(buckets) == 0:
            continue

        for ts in reversed(buckets):  ## Iterate through the timestamps in the bucket in reverse order
            if ts > window_start:
                if not halved:
                    one_count += size / 2  
                    halved = True
                else:
                    one_count += size 
            else:
                break  
            
    return math.ceil(one_count)

In [30]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [31]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 5
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     0.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 29
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     11.54 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 77
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     50.98 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 205
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     36.67 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 333
The true 

### 2. Bloom filters

In [32]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [33]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [34]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [ ]:
def generate_hash(h, N):
    hash_list = []
    primes = list(prime(i) for i in range(1, h + 1)) 
    random.shuffle(primes)  # Shuffle the list of primes
    
    for i in range(h):
        p = primes[i]  # Select a prime number for the hash function
        
        def hash_function(x, p = p):
            total = 0
            for idx, char in enumerate(x):
                total += (ord(char) * p ** (idx+1)) 
            return total % N   
        hash_list.append(hash_function)  # Append the hash function to the list
    
    return hash_list

Hash functions: [<function generate_hash.<locals>.hash_function at 0x0000027B88AB7D80>, <function generate_hash.<locals>.hash_function at 0x0000027B88AB7740>, <function generate_hash.<locals>.hash_function at 0x0000027B88AB7F60>]


In [45]:
hashes = generate_hash(h, bloom_size)

In [49]:
def create_bloom_filter(B, hashes, data):
    with data.open(encoding="utf-8") as f:
        for name in f:
            name = name.strip()
            for hash_function in hashes:
                index = hash_function(name)  
                B[index] = 1            
            
    return B

In [50]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [51]:
bloom_array

array([1., 0., 0., ..., 1., 0., 0.])

#### 2.2. Verify usernames

In [ ]:
def single_verify_username(bloom_array, hashes, new_user):
    for hash_fn in hashes:
        index = hash_fn(new_user)
        if bloom_array[index] == 0:
            return 0 
    return 1  

In [61]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [62]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [63]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [67]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open(encoding="utf-8") as f:
        for name in f:
            name = name.strip()
            total_name += 1
            if single_verify_username(bloom_array, hashes, name) == 1:
                taken_name += 1
            
    return round(taken_name/total_name*100,2)   

In [68]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 18.2%
----------------------------------------------------------


### 3. Flajolet-Martin

In [ ]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    hash_func = lambda x: (6 * x + 1) % 5

    for item in input_stream:
        h = hash_func(item)

        binary_h = bin(h)[2:]
        rightmost_zero_pos = len(binary_h) - len(binary_h.rstrip('0'))

        if rightmost_zero_pos > R:
            R = rightmost_zero_pos
    

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [94]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [ ]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [96]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [ ]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    
    
    
    return revenue

In [98]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 0
Trial 2 - Revenue generated: 0
Trial 3 - Revenue generated: 0
Trial 4 - Revenue generated: 0
Trial 5 - Revenue generated: 0
Trial 6 - Revenue generated: 0
Trial 7 - Revenue generated: 0
Trial 8 - Revenue generated: 0
Trial 9 - Revenue generated: 0
Trial 10 - Revenue generated: 0
------------------------------------------------
Average revenue generated for all trials:  0.0


#### 4.2. Balance Algorithm

In [99]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    
    return revenue

In [100]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 0
Trial 2 - Revenue generated: 0
Trial 3 - Revenue generated: 0
Trial 4 - Revenue generated: 0
Trial 5 - Revenue generated: 0
Trial 6 - Revenue generated: 0
Trial 7 - Revenue generated: 0
Trial 8 - Revenue generated: 0
Trial 9 - Revenue generated: 0
Trial 10 - Revenue generated: 0
-------------------------------------------
Average revenue generated for all trials:  0.0


### 5. Recommender System

In [101]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [102]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    
    return prediction   

In [103]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [104]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------


NameError: name 'prediction' is not defined

#### 5.2. Item-Item Collaborative Filtering

In [ ]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [ ]:
# Enter answer here

#### Case 2

In [ ]:
# Enter answer here

#### Case 3

In [ ]:
# Enter answer here

#### Case 4

In [ ]:
# Enter answer here

#### Case 5

In [ ]:
# Enter answer here